In [303]:
#CELL 1
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import pickle
import joblib




In [304]:
#CELL 2
df=pd.read_csv("C:/Users/Windows 10 Pro/Desktop/housedata/data/housing_dataset_1.csv")
df.head()


,ZIP,images_count,SQUARE_FT,parking_lot,rooms,user_id,phone,price,year_built,Yr_sold,sale_type
0,16508550,12,1300.236407,2,3,1716845,784856045,"KSh 130,000",2010,2008.0,WD
1,15715984,6,1275.000000,3,4,5078665,713739556,"KSh 150,000",2011,2007.0,WD
2,16387265,17,933.159722,4,5,1927270,707875750,"KSh 30,000",2013,2008.0,WD
3,16545942,10,929.921143,2,4,4365441,706098348,"KSh 38,000",2022,2006.0,WD
4,16028693,14,999.009247,1,6,236839,727591037,"KSh 130,000",2024,2008.0,WD


In [305]:
#CELL 3
df.info() # there is no missing value


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6280 entries, 0 to 6279
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ZIP           6280 non-null   int64  
 1   images_count  6280 non-null   int64  
 2   SQUARE_FT     6280 non-null   float64
 3   parking_lot   6280 non-null   object 
 4   rooms         6280 non-null   object 
 5   user_id       6280 non-null   int64  
 6   phone         6280 non-null   int64  
 7   price         6280 non-null   object 
 8   year_built    6280 non-null   object 
 9   Yr_sold       1460 non-null   float64
 10  sale_type     1460 non-null   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 539.8+ KB


In [306]:
#CELL4
df.dtypes


ZIP               int64
images_count      int64
SQUARE_FT       float64
parking_lot      object
rooms            object
user_id           int64
phone             int64
price            object
year_built       object
Yr_sold         float64
sale_type        object
dtype: object

In [307]:
#CELL 5
df.shape


(6280, 11)

In [308]:
#CELL 6
len(df[df['price']!=0])


6280

In [309]:
#cELL 7
df=df[df['price']!=0]
df.head()


,ZIP,images_count,SQUARE_FT,parking_lot,rooms,user_id,phone,price,year_built,Yr_sold,sale_type
0,16508550,12,1300.236407,2,3,1716845,784856045,"KSh 130,000",2010,2008.0,WD
1,15715984,6,1275.000000,3,4,5078665,713739556,"KSh 150,000",2011,2007.0,WD
2,16387265,17,933.159722,4,5,1927270,707875750,"KSh 30,000",2013,2008.0,WD
3,16545942,10,929.921143,2,4,4365441,706098348,"KSh 38,000",2022,2006.0,WD
4,16028693,14,999.009247,1,6,236839,727591037,"KSh 130,000",2024,2008.0,WD


In [310]:
#CELL 8
df.drop([ 'ZIP', 'images_count', 'SQUARE_FT',
         'parking_lot', 'rooms', 'user_id', 'phone', 
         'price', 'year_built', 'Yr_sold', 'sale_type'],axis = 1, inplace=True)



In [311]:
# CELL 9
feature_columns = ['ZIP', 'images_count', 'SQUARE_FT',
         'parking_lot', 'rooms', 'user_id', 'phone', 
         'price', 'year_built', 'Yr_sold', 'sale_type']



In [327]:
#CELL 10
df.head()


,ZIP,images_count,SQUARE_FT,parking_lot,rooms,user_id,phone,price,year_built,Yr_sold,sale_type
0,16508550,12,1300.236407,2,3,1716845,784856045,130000.0,2010,2008.0,1
1,15715984,6,1275.000000,3,4,5078665,713739556,150000.0,2011,2007.0,1
2,16387265,17,933.159722,4,5,1927270,707875750,30000.0,2013,2008.0,1
3,16545942,10,929.921143,2,4,4365441,706098348,38000.0,2022,2006.0,1
4,16028693,14,999.009247,1,6,236839,727591037,130000.0,2024,2008.0,1


In [313]:
#CELL 11
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Sample data corrected for equal length of lists
data = {
    'ZIP': [16508550, 15715984, 16387265, 16545942, 16028693, 15658958, 16581464, 16602649],
    'images_count': [12, 6, 17, 10, 14, 8, 5, 11],
    'SQUARE_FT': [1300.236407, 1275, 933.1597222, 929.9211427, 999.009247, 1100, 1200, 1150],
    'parking_lot': [2, 3, 4, 2, 1, 3, 2, 4],
    'rooms': [3, 4, 5, 4, 6, 5, 6, 7],
    'user_id': [1716845, 5078665, 1927270, 4365441, 236839, 123456, 654321, 987654],
    'phone': [784856045, 713739556, 707875750, 706098348, 727591037, 912345678, 987654321, 876543219],
    'price': ['KSh 130,000', 'KSh 150,000', 'KSh 30,000', 'KSh 38,000', 'KSh 130,000', 'KSh 120,000', 'KSh 95,000', 'KSh 105,000'],
    'year_built': ['2010', '2011', '2013', '2022', '2024', '2015', '2018', '2020'],
    'Yr_sold': [2008.0, 2007.0, 2008.0, 2006.0, 2008.0, 2010.0, 2012.0, 2014.0],
    'sale_type': ['WD', 'WD', 'WD', 'WD', 'WD', 'Lease', 'Lease', 'WD']
}

# Creating DataFrame
df = pd.DataFrame(data)

# Initialize the label encoder
label_encoder = LabelEncoder()

# Apply label encoding to categorical columns
df['sale_type'] = label_encoder.fit_transform(df['sale_type'])

# Convert 'year_built' to numeric
df['year_built'] = pd.to_numeric(df['year_built'], errors='coerce')

# Display the first few rows of the updated DataFrame
print(df.head())


        ZIP  images_count    SQUARE_FT  parking_lot  rooms  user_id  \
0  16508550            12  1300.236407            2      3  1716845   
1  15715984             6  1275.000000            3      4  5078665   
2  16387265            17   933.159722            4      5  1927270   
3  16545942            10   929.921143            2      4  4365441   
4  16028693            14   999.009247            1      6   236839   

       phone        price  year_built  Yr_sold  sale_type  
0  784856045  KSh 130,000        2010   2008.0          1  
1  713739556  KSh 150,000        2011   2007.0          1  
2  707875750   KSh 30,000        2013   2008.0          1  
3  706098348   KSh 38,000        2022   2006.0          1  
4  727591037  KSh 130,000        2024   2008.0          1  


In [314]:
#CELL 12
y = df['price']
x = df.drop(['price'],axis=1)


In [315]:
# CELL 13
df['price'] = df['price'].str.replace('KSh', '').str.replace(',', '').astype(float)
y = df['price']


In [316]:
# CELL 14
numeric_columns = ['ZIP', 'images_count', 'SQUARE_FT', 'parking_lot', 'rooms', 'user_id', 'phone', 'price', 'year_built', 'Yr_sold', 'sale_type'
 ]
X = df[numeric_columns]


In [317]:

#CELL 15
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler



In [318]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)


In [319]:
X = df.drop(columns=['price'])
y = df['price']

In [320]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Create a DataFrame from the scaled data with the correct column names
x_train_df = pd.DataFrame(x_train, columns=X.columns)
x_test_df = pd.DataFrame(x_test, columns=X.columns)

print(x_train_df.head())



        ZIP  images_count  SQUARE_FT  parking_lot     rooms   user_id  \
0 -1.170389     -1.597191   1.110883     0.522233 -0.620174  1.536975   
1  1.081599      0.319438   0.180841     1.566699  1.612452 -0.563259   
2  0.937572     -0.063888  -1.456619    -0.522233 -0.620174  1.170822   
3  0.842602      0.702764   1.298650    -0.522233 -1.364382 -0.188908   
4 -1.315226     -0.830540  -0.191175     0.522233  0.124035 -1.006919   

      phone  year_built   Yr_sold  sale_type  
0 -0.904941   -1.120631 -0.702764   0.447214  
1  1.109859    0.560316  1.980517   0.447214  
2 -0.999506    0.933859 -1.086090   0.447214  
3 -0.024829   -1.307403 -0.319438   0.447214  
4  1.552937   -0.373544  0.447214  -2.236068  


In [321]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [322]:

# Handling missing values if necessary
X = X.fillna(X.mean())
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)



In [323]:
print(x_train_df.head())


        ZIP  images_count  SQUARE_FT  parking_lot     rooms   user_id  \
0 -1.170389     -1.597191   1.110883     0.522233 -0.620174  1.536975   
1  1.081599      0.319438   0.180841     1.566699  1.612452 -0.563259   
2  0.937572     -0.063888  -1.456619    -0.522233 -0.620174  1.170822   
3  0.842602      0.702764   1.298650    -0.522233 -1.364382 -0.188908   
4 -1.315226     -0.830540  -0.191175     0.522233  0.124035 -1.006919   

      phone  year_built   Yr_sold  sale_type  
0 -0.904941   -1.120631 -0.702764   0.447214  
1  1.109859    0.560316  1.980517   0.447214  
2 -0.999506    0.933859 -1.086090   0.447214  
3 -0.024829   -1.307403 -0.319438   0.447214  
4  1.552937   -0.373544  0.447214  -2.236068  


In [324]:
model = LinearRegression()
model.fit(x_train,y_train)
Y_pred = model.predict(x_test)


In [325]:
from sklearn.metrics import r2_score
y_test_pre=model.predict(x_test)
test_acc=r2_score(y_test,y_test_pre)
test_acc


-0.6795007499250254

In [326]:
model_path = 'C:/Users/Windows 10 Pro/Desktop/housedata/models/house_price_model.pkl'
scaler_path = 'C:/Users/Windows 10 Pro/Desktop/housedata/models/scaler.pkl'

joblib.dump(model, model_path)
joblib.dump(scaler, scaler_path)

print("Model and scaler saved successfully.")



Model and scaler saved successfully.
